In [4]:
import pennylane as qml
from pennylane import numpy as np
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
import time

# 1. Load and preprocess MNIST dataset (using sklearn's load_digits for 8x8 images)
digits = load_digits()
X, y = digits.data, digits.target

# Filter to binary classification (e.g., digits 0 and 1) for simplicity
binary_classes = [0, 1]
mask = np.isin(y, binary_classes)
X, y = X[mask], y[mask]

# Reduce dataset size for quantum kernel feasibility (e.g., 100 training, 20 test)
n_train = 200
n_test = 100
indices = np.random.permutation(len(X))
X_train, y_train = X[indices[:n_train]], y[indices[:n_train]]
X_test, y_test = X[indices[n_train:n_train + n_test]], y[indices[n_train:n_train + n_test]]

# Apply PCA to reduce dimensionality (e.g., to 2 features for a 2-qubit circuit)
n_features = 2
pca = PCA(n_components=n_features)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

# Normalize features
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 2. Define the device (simulator; can use qiskit.aer with pennylane-qiskit)
dev = qml.device('default.qubit', wires=n_features)

# 3. Define the ZZFeatureMap equivalent (ansatz for embedding classical features)
def feature_map(x, wires):
    for i in range(len(wires)):
        qml.RY(2 * np.pi * x[i], wires=wires[i])  # Rotation based on features
    qml.CNOT(wires=[wires[0], wires[1]])  # Entangling gate (similar to ZZ interaction)

# 4. Define the quantum kernel function (computes fidelity-like overlap)
@qml.qnode(dev)
def kernel_circuit(x1, x2):
    feature_map(x1, wires=range(n_features))
    qml.adjoint(feature_map)(x2, wires=range(n_features))
    return qml.probs(wires=range(n_features))

def quantum_kernel(x1, x2):
    return kernel_circuit(x1, x2)[0]  # Probability of |0...0> state (fidelity)

# 5. Compute the kernel matrix for training data
n_samples = len(X_train_scaled)
kernel_matrix = np.zeros((n_samples, n_samples))
print("Computing training kernel matrix...")
start_time = time.time()
for i in range(n_samples):
    for j in range(n_samples):
        kernel_matrix[i, j] = quantum_kernel(X_train_scaled[i], X_train_scaled[j])
print(f"Training kernel matrix computed in {time.time() - start_time:.2f} seconds")

# 6. Train the QSVC using scikit-learn's SVC with the quantum kernel
qsvc = SVC(kernel='precomputed')
qsvc.fit(kernel_matrix, y_train)

# 7. Compute kernel matrix for test data
test_kernel_matrix = np.zeros((len(X_test_scaled), n_samples))
print("Computing test kernel matrix...")
start_time = time.time()
for i in range(len(X_test_scaled)):
    for j in range(n_samples):
        test_kernel_matrix[i, j] = quantum_kernel(X_test_scaled[i], X_train_scaled[j])
print(f"Test kernel matrix computed in {time.time() - start_time:.2f} seconds")

# 8. Make predictions and evaluate
y_pred = qsvc.predict(test_kernel_matrix)
accuracy = accuracy_score(y_test, y_pred)
print("Predictions:", y_pred)
print("True labels:", y_test)
print(f"Test accuracy: {accuracy:.4f}")

Computing training kernel matrix...
Training kernel matrix computed in 61.98 seconds
Computing test kernel matrix...
Test kernel matrix computed in 32.46 seconds
Predictions: [0 1 0 1 1 0 0 1 1 1 0 1 1 1 0 0 1 0 1 0 1 0 0 0 1 0 1 1 1 1 1 0 1 1 0 1 0
 1 1 0 0 1 0 1 1 0 1 1 0 1 0 1 1 1 0 0 1 1 1 1 1 1 1 0 0 0 0 0 1 1 0 1 0 0
 1 1 0 1 0 0 0 0 1 0 0 0 0 1 1 0 1 1 1 1 1 1 0 0 0 0]
True labels: [1 0 1 0 0 0 1 0 0 1 0 0 1 1 1 0 1 0 1 0 0 1 0 1 0 1 0 1 0 1 1 0 1 0 1 0 1
 1 1 1 1 0 0 0 1 0 1 1 0 0 0 1 1 1 0 0 0 1 0 1 1 1 0 0 0 1 0 1 0 1 0 0 0 0
 0 1 0 0 1 1 1 0 1 1 0 0 0 1 1 0 0 0 1 1 0 0 0 0 0 0]
Test accuracy: 0.5700
